In [15]:
import os
import collections as col
import dnaio

assembly_path = '/home/local/work/pipeline/run_folder/references/assemblies'
assemblies = ['T2Tv1_38p13Y_chm13.fasta', 'GRCh38_HGSVC2_noalt.fasta']

roi_path = '/home/local/work/data/hgsvc/roi'
roi_files = ['roi-t2t_1p3613.bed', 'roi_HLA_3q29_HLAn_1p3613.bed']

out_path = '/home/local/work/data'
outfiles = ['t2t_', 'hg38_']

for out_prefix, roi_file, assm_file in zip(outfiles, roi_files, assemblies):
    assm_path = os.path.join(assembly_path, assm_file)
    bed_file = os.path.join(roi_path, roi_file)
    
    regions = col.defaultdict(list)
    
    with open(bed_file, 'r') as bed:
        for line in bed:
            if ('1p36.13' not in line) and ('3q29' not in line):
                continue
            chrom, start, end, name = line.strip().split()
            regions[chrom].append((int(start), int(end), name))
            
    with dnaio.open(assm_path) as fasta:
        for record in fasta:
            if record.name not in regions:
                continue
            for (start, end, name) in regions[record.name]:
                
                start_lower = int(start // 1e6 * 1e6)
                end_upper = int(end // 1e6 * 1e6 + 1e6)
                region_seq = record.sequence[start_lower:end_upper]
                                
                out_file_name = out_prefix + '{}_{}M-{}M_{}.fasta'.format(
                    record.name,
                    int(start_lower // 1e6),
                    int(end_upper // 1e6),
                    name
                )
                out_file_path = os.path.join(out_path, out_file_name)
                writer = dnaio.FastaWriter(out_file_path, 120)
                writer.write(out_file_name.rsplit('.', 1)[0], region_seq)
                
